# random forest kobe bryant

This project is inspired by Siraj Raval's [Video](https://www.youtube.com/watch?v=QHOazyP-YlM) and [notebook](https://github.com/llSourcell/random_forests/blob/master/Random%20Forests%20.ipynb) about random forests.

## gini index

The gini index calculates a score for a split, that was done on the data. That score gives an idea how mixed the resulting groups are. If instances of only one category are in the group, the gini index is zero.

The gini index of one group is calculated like this:

```python
gini = 1
for c in classes:
    count = group['class'].count(c)
    gini -= (count * count) / (len(group) * len(group))
```

# coding

In [1]:
import pandas as pd
import numpy as np
np.random.seed(0)

## data processing

In [2]:
data = pd.read_csv('data.csv')

In [3]:
# let's only take the data which is labeled
data = data[pd.notna(data['shot_made_flag'])]
data.rename(columns={'shot_made_flag': 'class'},
            inplace=True)
data.drop('shot_id', axis=1, inplace=True)
data.head()

,action_type,combined_shot_type,game_event_id,game_id,lat,loc_x,loc_y,lon,minutes_remaining,period,...,class,shot_type,shot_zone_area,shot_zone_basic,shot_zone_range,team_id,team_name,game_date,matchup,opponent
1,Jump Shot,Jump Shot,12,20000012,34.0443,-157,0,-118.4268,10,1,...,0.0,2PT Field Goal,Left Side(L),Mid-Range,8-16 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR
2,Jump Shot,Jump Shot,35,20000012,33.9093,-101,135,-118.3708,7,1,...,1.0,2PT Field Goal,Left Side Center(LC),Mid-Range,16-24 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR
3,Jump Shot,Jump Shot,43,20000012,33.8693,138,175,-118.1318,6,1,...,0.0,2PT Field Goal,Right Side Center(RC),Mid-Range,16-24 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR
4,Driving Dunk Shot,Dunk,155,20000012,34.0443,0,0,-118.2698,6,2,...,1.0,2PT Field Goal,Center(C),Restricted Area,Less Than 8 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR
5,Jump Shot,Jump Shot,244,20000012,34.0553,-145,-11,-118.4148,9,3,...,0.0,2PT Field Goal,Left Side(L),Mid-Range,8-16 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR


In [4]:
# important global dict for distinguishing categorical and numerical columns
feature_type = {'action_type': 'cat',
                'combined_shot_type': 'cat',
                'game_event_id': 'num',
                'game_id': 'num',
                'lat': 'num',
                'loc_x': 'num',
                'loc_y': 'num',
                'lon': 'num',
                'minutes_remaining': 'num',
                'period': 'cat',
                'playoffs': 'cat',
                'season': 'cat',
                'seconds_remaining': 'num',
                'shot_distance': 'num',
                'shot_type': 'cat',
                'shot_zone_area': 'cat',
                'shot_zone_range': 'cat',
                'shot_zone_basic': 'cat',
                'team_id': 'cat',
                'team_name': 'cat',
                'game_date': 'cat',
                'matchup': 'cat',
                'opponent': 'cat',}

## helper functions

In [5]:
def subsample(df, sample_size, return_indexes=False):
    """
    returns a random subsample of the dataframe with size sample_size
    df: pd.DataFrame
    sample_size: integer
    return: pd.DataFrame
    """
    indexes = np.random.choice(df.index, sample_size)
    if return_indexes:
        return df.loc[indexes], indexes
    else:
        return df.loc[indexes]

def cross_validation_split(df, n_folds):
    """
    returns n_folds dataframes out of df with equal length
    df: pd.DataFrame
    n_folds: integer
    return: list of pd.DataFrame
    """
    dfs = []
    fold_size = int(len(df) / n_folds)
    for i in range(n_folds):
        fold, indexes = subsample(df, fold_size, return_indexes=True)
        dfs.append(fold)
        df.drop(indexes, inplace=True)
    return dfs

def split_by(df, feature, value, is_numerical=True):
    """
    make the split of a decision tree node
    df: pd.DataFrame
    feature: column name (mostly string)
    value: any
    is_numerical: bool
    return: (pd.DataFrame, pd.DataFrame)
    """
    if is_numerical:
        return df[df[feature] < value], df[df[feature] >= value]
    else:
        # categorical feature
        return df[df[feature] == value], df[df[feature] != value]
        
def accuracy(predicted, target):
    """
    return percentage of correctly predicted cases
    predicted: np.array
    target: np.array
    return: float between 0 and 1
    """
    assert len(predicted) == len(target)
    return (predicted == target).sum() / len(predicted)

def most_frequent(array):
    """
    return value that is the most frequent
    array: np.array
    """
    unique, counts = np.unique(array, return_counts=True)
    return unique[np.argmax(counts)]

## gini index

In [112]:
def gini_index(splits, info=False):
    """
    calculate weighted gini index of splits
    splits: list of pd.DataFrame (normally of length 2)
    info: bool (if True, print)
    """
    overall_gini = 0
    len_sum = sum([len(s) for s in splits])
    for df in splits:
        if info:
            print(f'len: {len(df)}')
        gini = 1
        l = len(df)
        for c in np.unique(df['class'].to_numpy()):
            count = (df['class'] == c).sum()
            gini -= (count * count) / (l*l)
            if info:
                print(f'c: {c}, count: {count}')
        if info:
            print(f'gini: {gini}')
        overall_gini += gini * (len(df)/len_sum)
    return overall_gini

## OOP Node

In [160]:
class Node:
    """
    This class is the Class which builds the Tree,
    In the init function it searches the best split 
    possible for the given (branch) of data by using
    the gini index, if the depth isn't at the maximum
    yet, it builds it's childs
    attributes:
    feat: string
    value: any
    num: bool
    left: Node or class
    right: Node or class
    """
    def __init__(self, data, depth, max_depth, min_size, info=False):
        """
        find best possible split for the data
        data: pd.DataFrame
        depth: integer
        max_depth: integer
        min_size: integer
        info: bool (if True print info)
        """
        if info:
            print(f'===== Node depth: {depth}, datalength: {len(data)} =====')
        best = None
        # find best split
        feature_values = {}
        for i, (ind, row) in enumerate(data.iterrows()):
            if info and i % 100 == 0:
                print(f'{i}th row')
            for feat in data.columns:
                if feat == 'class':
                    # this is not a feature, it's the class
                    continue
                num = feature_type[feat] == 'num'
                val = row[feat]
                if not num and not feature_values.get(feat, None):
                    feature_values[feat] = []
                # test split only once for every value of a categorical feature
                if not num and val not in feature_values[feat]:
                    feature_values[feat].append(val)
                elif not num:
                    continue
                    
                # do the split by the test
                # split_by(data, feature, value, is_numerical)
                left, right = split_by(data, feat, val, num)
                if len(left) == 0 or len(right) == 0:
                    if not best: # prevent having no split at all:
                        best = {
                            'gini': 1.0,
                            'feature': feat,
                            'value': val,
                            'left': left,
                            'right': right
                        }
                    continue # don't make splits with no split
                # calculate score of this split
                gini = gini_index([left, right])
                if not best or gini < best['gini']:
                    best = {
                        'gini': gini,
                        'feature': feat,
                        'value': val,
                        'left': left,
                        'right': right
                    }
                    if info:
                        print(f'gini: {gini}, test: {feat} {"<" if num else "=="} {val}')
                    
        self.feat = best['feature']
        self.value = best['value']
        self.num = feature_type[self.feat] == 'num'
        left, right = best['left'], best['right']
        
        # edge cases
        if left.empty or right.empty:
            class_array = np.concatenate([left['class'].to_numpy(), right['class'].to_numpy()])
            self.left = self.right = most_frequent(class_array)
        elif depth >= max_depth:
            self.left = most_frequent(left['class'].to_numpy())
            self.right = most_frequent(right['class'].to_numpy())
        else:
            if len(left) <= min_size:
                self.left = most_frequent(left['class'].to_numpy())
            else:
                # child node left
                self.left = Node(left, depth+1, max_depth, min_size, info=info)
            if len(right) <= min_size:
                self.right = most_frequent(right['class'].to_numpy())
            else:
                self.right = Node(right, depth+1, max_depth, min_size, info=info)           
        
    def predict(self, row):
        """
        check row on my split, 
        then send it to child node,
        or return class if no child
        row: pd.Series / pd.DataFrame
        """
        if isinstance(row, pd.Series):
            if (self.num and row[self.feat] < self.value) or (not self.num and row[self.feat] == self.value):
                # left is for those with smaller value if numerical
                # or for those with equal category in feature if categorical
                if isinstance(self.left, Node):
                    return self.left.predict(row)
                else:
                    return self.left
            else:
                if isinstance(self.right, Node):
                    return self.right.predict(row)
                else:
                    return self.right
        elif isinstance(row, pd.DataFrame):
            predictions = np.array([])
            left_rows = row[row[self.feat] < self.value] if self.num else row[row[self.feat] == self.value]
            if isinstance(self.left, Node):
                predictions = np.append(predictions, self.left.predict(left_rows), axis=0)
            else:
                predictions = np.append(predictions, np.full((len(left_rows)), self.left), axis=0)
                              
            right_rows = row[row[self.feat] >= self.value] if self.num else row[row[self.feat] != self.value]
            if isinstance(self.right, Node):
                predictions = np.append(predictions, self.right.predict(right_rows), axis=0)
            else:
                predictions = np.append(predictions, np.full((len(right_rows)), self.right), axis=0)
            
            return predictions
                              
    def __str__(self):
        return f'{self.feat} {"<" if self.num else "=="} {self.value}'

    def show(self):
        """
        prints out the structure of the tree
        """
        # make dict of key being a level of height in the print 
        # and the value being a tuple of string and int, with being the depth
        index = {0: str(self)}
        current = [(self, 0)]
        next_ = []
        depth = 0
        while len(current):
            depth += 1
            strs = []
            for node, lvl in current:
                if isinstance(node, Node):
                    index[lvl] = (str(node), depth)
                    step = 2**depth
                    next_.append((node.left, lvl + 1 / step))
                    next_.append((node.right, lvl - 1 / step))
                else:
                    index[lvl] = (f'class: {node}', depth)
            current = next_
            next_ = []
        
        for lvl in sorted(index.keys()):
            s, depth = index[lvl]
            # twelve is max length of 
            print(f'{(8 * (depth-1)) * " "}{s}')

## RandomForest

In [172]:
class RandomForest:
    """
    random forest of certain number of nodes (tree roots)
    which are decision trees built out of random subsample
    of data
    """
    def __init__(self, train, n_trees, max_depth, min_node_size, sample_size, info=False, node_info=False):
        """
        create n_trees roots (trees)
        train: pd.DataFrame
        n_trees: integer
        max_depth: integers
        """
        self.trees = []
        for i in range(n_trees):
            if info:
                print(f'{i}th tree')
            self.trees.append(Node(subsample(train, sample_size), 1, max_depth, min_node_size, info=node_info))
        
    def predict(self, row, return_all=False):
        predictions = np.array([tree.predict(row) for tree in self.trees])
        if isinstance(row, pd.Series):
            if return_all:
                return most_frequent(predictions), predictions
            else:
                return most_frequent(predictions)
        elif isinstance(row, pd.DataFrame):
            # row i are all predictions of tree i
            # column i are all predictions for the ith row in the df (named row)
            predictions = np.array([tree.predict(row) for tree in self.trees])
            # np.array of the prediction for every row in the df (named row)
            results = np.array([most_frequent(predictions[:, i]) for i in range(len(predictions[0]))])
            if return_all:
                return results, predictions
            else:
                return results

In [173]:
folds = cross_validation_split(data, 10)
train = pd.concat(folds[:-1])

forest = RandomForest(train, 10, 10, 1, 5000, info=True)

0th tree
1th tree
2th tree
3th tree
4th tree
5th tree
6th tree
7th tree


KeyboardInterrupt: 

In [ ]:
p = forest.predict(test)
accuracy(p, test['class'].to_numpy())